Zadanie

Utwórz model prognostyczny klasyfikacji jakości powietrza na następujące klasy zdefiniowane koncentracją PM10 na podstawie danych  PM10Krakow_do10st.csv, zastosuj algorytm Decision Tree i Random Forest (wybierz lepszy)

(opis kod zakres)

1. bardzo dobra   0   0-20
2. dobra          1   20-60
3. umiarkowana    2   60-100
4. dostateczna    3   100-140
5. zła            4   140-200
6. bardzo zła     5   pow. 200

Jaką dokładność daje model co do wyznaczenia prawdziwej klasy, jaką co do  wyznaczenia prawdziwej klasy +-1

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as spstats
from sklearn.ensemble import RandomForestRegressor
import seaborn as sns

In [2]:
pm10=pd.read_csv('PM10Krakow_do10st.csv')
pm10.head(5)

,Code,Concentration,TimeSM,T,H%,Wind_dir,Wind_vel,UR200,WA200,GR200,...,CorIN2,CorOUT2,CorIN6,CorOUT6,CorIN10,CorOUT10,CorIN15,CorOUT15,CorIN30,CorOUT30
0,PL0012A,254.931,2015-12-31 23:00:00,-10.3,92.0,"Calm, no wind",0.0,60.41,0.0,0.0,...,0.8975,0.8975,0.56875,0.56875,0.37875,0.37875,0.26875,0.26875,0.1675,0.1675
1,PL0012A,303.558,2016-01-01 00:00:00,-10.6,93.0,"Calm, no wind",0.0,60.41,0.0,0.0,...,0.8975,0.8975,0.56875,0.56875,0.37875,0.37875,0.26875,0.26875,0.1675,0.1675
2,PL0012A,289.828,2016-01-01 01:00:00,-11.0,92.0,"Calm, no wind",0.0,60.41,0.0,0.0,...,0.8975,0.8975,0.56875,0.56875,0.37875,0.37875,0.26875,0.26875,0.1675,0.1675
3,PL0012A,279.998,2016-01-01 02:00:00,-11.0,93.0,Wind blowing from the east-northeast,1.0,60.41,0.0,0.0,...,0.7950,0.8500,0.84000,0.59000,0.66500,0.42000,0.50500,0.37000,0.4100,0.1900
4,PL0012A,289.167,2016-01-01 03:00:00,-11.6,93.0,Wind blowing from the north-east,1.0,60.41,0.0,0.0,...,0.6300,0.9100,0.69000,0.75000,0.34000,0.50000,0.15000,0.52000,0.0400,0.2300


In [3]:
pm10.describe()

,Concentration,T,H%,Wind_vel,UR200,WA200,GR200,IN200,RO200,UR600,...,CorIN2,CorOUT2,CorIN6,CorOUT6,CorIN10,CorOUT10,CorIN15,CorOUT15,CorIN30,CorOUT30
count,11661.000000,11661.000000,11661.000000,11661.000000,1.166100e+04,11661.0,11661.0,1.166100e+04,1.166100e+04,11661.00,...,11661.000000,11661.000000,11661.000000,11661.000000,11661.000000,11661.000000,11661.000000,11661.000000,11661.000000,11661.000000
mean,72.953937,2.083269,85.071435,3.240117,6.041000e+01,0.0,0.0,9.710000e+00,2.316000e+01,35.13,...,0.841325,0.855445,0.591349,0.679055,0.380833,0.473271,0.288253,0.368492,0.157655,0.238917
std,56.264132,5.332912,11.961178,2.359494,1.421146e-14,0.0,0.0,3.552866e-15,7.105732e-15,0.00,...,0.105259,0.110705,0.166071,0.177732,0.167615,0.233328,0.183150,0.233609,0.147118,0.227392
min,3.000000,-23.500000,25.000000,0.000000,6.041000e+01,0.0,0.0,9.710000e+00,2.316000e+01,35.13,...,0.630000,0.630000,0.190000,0.190000,0.070000,0.070000,0.030000,0.030000,0.010000,0.010000
25%,35.058100,-0.800000,79.000000,1.000000,6.041000e+01,0.0,0.0,9.710000e+00,2.316000e+01,35.13,...,0.795000,0.795000,0.440000,0.568750,0.340000,0.340000,0.150000,0.150000,0.040000,0.040000
50%,57.936000,2.800000,88.000000,3.000000,6.041000e+01,0.0,0.0,9.710000e+00,2.316000e+01,35.13,...,0.850000,0.897500,0.590000,0.690000,0.345000,0.390000,0.268750,0.300000,0.150000,0.167500
75%,93.102300,6.100000,94.000000,5.000000,6.041000e+01,0.0,0.0,9.710000e+00,2.316000e+01,35.13,...,0.910000,0.950000,0.690000,0.840000,0.420000,0.665000,0.370000,0.520000,0.190000,0.410000
max,444.367000,10.000000,100.000000,16.000000,6.041000e+01,0.0,0.0,9.710000e+00,2.316000e+01,35.13,...,1.000000,1.000000,0.990000,0.990000,0.990000,0.990000,0.860000,0.860000,0.780000,0.780000


In [4]:
# Przypisanie nowej wartości za pomocą funkcji

def label6(x):
    if x<=20:
        return 0
    elif x<=60:
        return 1
    elif x<=100:
        return 2
    elif x<=140:
        return 3
    elif x<=200:
        return 4
    else:
        return 5
    
pm10['label']=pm10['Concentration'].apply(label6) 

In [5]:
pm10.head(5)

,Code,Concentration,TimeSM,T,H%,Wind_dir,Wind_vel,UR200,WA200,GR200,...,CorOUT2,CorIN6,CorOUT6,CorIN10,CorOUT10,CorIN15,CorOUT15,CorIN30,CorOUT30,label
0,PL0012A,254.931,2015-12-31 23:00:00,-10.3,92.0,"Calm, no wind",0.0,60.41,0.0,0.0,...,0.8975,0.56875,0.56875,0.37875,0.37875,0.26875,0.26875,0.1675,0.1675,5
1,PL0012A,303.558,2016-01-01 00:00:00,-10.6,93.0,"Calm, no wind",0.0,60.41,0.0,0.0,...,0.8975,0.56875,0.56875,0.37875,0.37875,0.26875,0.26875,0.1675,0.1675,5
2,PL0012A,289.828,2016-01-01 01:00:00,-11.0,92.0,"Calm, no wind",0.0,60.41,0.0,0.0,...,0.8975,0.56875,0.56875,0.37875,0.37875,0.26875,0.26875,0.1675,0.1675,5
3,PL0012A,279.998,2016-01-01 02:00:00,-11.0,93.0,Wind blowing from the east-northeast,1.0,60.41,0.0,0.0,...,0.8500,0.84000,0.59000,0.66500,0.42000,0.50500,0.37000,0.4100,0.1900,5
4,PL0012A,289.167,2016-01-01 03:00:00,-11.6,93.0,Wind blowing from the north-east,1.0,60.41,0.0,0.0,...,0.9100,0.69000,0.75000,0.34000,0.50000,0.15000,0.52000,0.0400,0.2300,5


In [6]:
X=pm10[['T', 'Wind_vel', 'CorOUT6', 'CorIN6', 'H%']]
y=pm10[['label']]

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                    test_size=0.4, random_state=101)

#### Decision tree

In [8]:
from sklearn.tree import DecisionTreeClassifier

In [9]:
dtree = DecisionTreeClassifier()

In [10]:
clf =dtree.fit(X_train,y_train)

In [11]:
predictions = dtree.predict(X_test)

In [12]:
from sklearn.metrics import classification_report,confusion_matrix

In [13]:
print(confusion_matrix(y_test,predictions))

[[ 174  194   42   14    6    2]
 [ 202 1105  425  131   69   28]
 [  51  457  444  182   79   35]
 [  13  134  164  129   52   30]
 [   2   67   85   75   62   38]
 [   1   19   36   32   24   62]]


In [14]:
print(classification_report(y_test,predictions))

             precision    recall  f1-score   support

          0       0.39      0.40      0.40       432
          1       0.56      0.56      0.56      1960
          2       0.37      0.36      0.36      1248
          3       0.23      0.25      0.24       522
          4       0.21      0.19      0.20       329
          5       0.32      0.36      0.34       174

avg / total       0.42      0.42      0.42      4665



#### Random Forest

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, 
                                    np.ravel(y), 
                                    test_size=0.30, 
                                    random_state=102)

In [16]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=100)
fff=rfc.fit(X_train, y_train)

In [17]:
predictions = rfc.predict(X_test)
print(classification_report(y_test,predictions))

             precision    recall  f1-score   support

          0       0.52      0.39      0.44       339
          1       0.59      0.67      0.63      1494
          2       0.42      0.42      0.42       912
          3       0.27      0.20      0.23       398
          4       0.33      0.26      0.29       234
          5       0.46      0.54      0.50       122

avg / total       0.48      0.49      0.48      3499



In [18]:
print(confusion_matrix(y_test,predictions))

[[ 132  178   21    6    1    1]
 [ 111 1007  278   61   29    8]
 [  10  376  380   94   32   20]
 [   3   95  154   81   44   21]
 [   0   43   59   45   60   27]
 [   0    7   21   10   18   66]]


In [19]:
aa=confusion_matrix(y_test,predictions)

In [21]:
suma=0
for i in range (0,6):
    if i==0:
        klas_p=aa[i,:2].sum()/aa[i,:].sum()
        print (i,klas_p)
        suma=suma+klas_p
    elif i==5:
        klas_p=aa[i,-2:].sum()/aa[i,:].sum()
        print (i,klas_p)
        suma=suma+klas_p
    else:
         klas_p=aa[i,i-1:i+2].sum()/aa[i,:].sum()
         print(i,klas_p) 
         suma=suma+klas_p
print(suma/6)        

0 0.9144542772861357
1 0.9344042838018741
2 0.9320175438596491
3 0.7010050251256281
4 0.5641025641025641
5 0.6885245901639344
0.7890847140566309


In [34]:
suma=0
for i in range (0,6):
    if i==0:
        klas_p=aa[i,:2].sum()
        #print (i,klas_p)
        suma=suma+klas_p
    elif i==5:
        klas_p=aa[i,-2:].sum()
        #print (i,klas_p)
        suma=suma+klas_p
    else:
         klas_p=aa[i,i-1:i+2].sum()
         #print(i,klas_p) 
         suma=suma+klas_p
print(suma/aa.sum()) 

0.8719634181194627
